[三陰乳癌 analyzed by Apriori Method](#Note)

In [ ]:
import pandas as pd
import time

In [ ]:
# concer with L_{k-1}  to C_k 
def connect_string(x, ms):
  x = list([sorted(i.split(ms)) for i in x])
  l = len(x[0])
  r = []
  for i in range(len(x)):
    for j in range(i,len(x)):
      if x[i][:l-1] == x[j][:l-1] and x[i][l-1] != x[j][l-1]:
        r.append(x[i][:l-1]+sorted([x[j][l-1],x[i][l-1]]))
  return r

# accociation rules seeking
def find_rule(d, support, confidence, ms = '--'):
  # the output   
  result = pd.DataFrame(index=['support', 'confidence']) 
  
  # degree of Support computation
  support_series = 1.0*d.sum()/len(d)
  # the first filter for support
  column = list(support_series[support_series > support].index) 
  k = 0
  
  while len(column) > 1:
    k = k+1
    print('\n第 %s 次搜尋進行中...' %k)
    column = connect_string(column, ms)
    print('數目：%s...' %len(column))
    
    # calculate support-value
    sf = lambda i: d[i].prod(axis=1, numeric_only = True)
    d_2 = pd.DataFrame(list(map(sf,column)), index = [ms.join(i) for i in column]).T
    
    # update the support-value while connected to L
    support_series_2 = 1.0*d_2[[ms.join(i) for i in column]].sum()/len(d) 
    column = list(support_series_2[support_series_2 > support].index) 
    support_series = support_series.append(support_series_2)
    column2 = []
    
    # detect the rules and seek which one is right
    for i in column: 
      i = i.split(ms)
      for j in range(len(i)):
        column2.append(i[:j]+i[j+1:]+i[j:j+1])
    
    # introduce the array for confidence and compute
    cofidence_series = pd.Series(index=[ms.join(i) for i in column2]) 
    for i in column2: 
      cofidence_series[ms.join(i)] = support_series[ms.join(sorted(i))]/support_series[ms.join(i[:len(i)-1])]
    
    # validate the confidence
    for i in cofidence_series[cofidence_series > confidence].index:
      result[i] = 0.0
      result[i]['confidence'] = cofidence_series[i]
      result[i]['support'] = support_series[ms.join(sorted(i.split(ms)))]
  
  # output the result
  result = result.T.sort_values(['confidence','support'], ascending = False)
  print('\n结果為：')
  print(result)
  
  return result

In [ ]:
inputfile = '../data/apriori.csv'
data = pd.read_csv(inputfile, header=None, dtype = object)

In [ ]:
# timing the computational processing 
start = time.clock() 
print('\n轉換原始數據至 binary (0-1) 矩陣...')
# initial the test matrix
ct = lambda x : pd.Series(1, index = x[pd.notnull(x)]) #转换0-1矩阵的过渡函数
b = list(map(ct, data.as_matrix())) 
# replace the null columns by 0  
data = pd.DataFrame(b).fillna(0)
# time up
end = time.clock() 
print('\n轉換完成，執行時間：%0.2f秒' %(end-start))
del b 

In [ ]:
# the minima of default support and confidence levels
support = 0.06 #最小支持度
confidence = 0.75 #最小置信度
ms = '---'

start = time.clock() 
print('\n最小支持度 %s, 最小可信度 %s' %(support,confidence))
print('\n開始搜尋關聯規則...')
find_rule(data, support, confidence, ms)
end = time.clock()
print('\n搜尋完成，執行時間：%0.2f 秒' %(end-start))

Summary and Discussion
---
There are four rules induced here.
1. ```A3---F4---H4``` means A3, F4  → H4 and etc.
- Not all the rules are meaningful, only the last induced result in ```H``` cardinal.
- Since the first rule, ```A3---F4---H4```, got the maximal support and confidence too, this explains the fact:
      "TNM 分期" being H4 with confidence 88% and occurs possibly with 7.96%.
- Rule 2: ...
- Summary, the cancer with "TNM 分期" being H4 related to '肝腎陰虛症型 (F)', '肝氣鬱結症型 (A)', and '沖任失調症型 (C)', (<b style="color:red;">肝氣不舒鬱積而成</b>).

In [ ]:
datafile0 = '../data/data.csv'
df = pd.read_csv(datafile0) 

In [ ]:
df.head()

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

%matplotlib inline

In [ ]:
plt.hist?

In [ ]:
plt.scatter(df[['肝腎陰虛症型係數']],df[['肝氣鬱結症型係數']])

In [ ]:
A= plt.figure().gca(projection='3d')
A.scatter(df[['肝腎陰虛症型係數']],df[['肝氣鬱結症型係數']],df[['沖任失調症型係數']])